# LSTM Training

In [15]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [16]:
import pandas as pd
import numpy as np
import itertools
import os
import random

In [34]:
# read finished Data; case training
def readData(src, noConfFlag=True):
    dataseries = []
    for i in itertools.count():
        try:
            #print(i)
            dataseries.append(np.array(pd.read_json(src+ '/'+os.path.basename(src[:-4])+'_'+str(i).zfill(12)+'_keypoints.json').iloc[0,0]['pose_keypoints_2d']))
            if noConfFlag:
                dataseries[-1] = np.append(dataseries[-1][:2],np.delete(dataseries[-1][2:], slice(None, None, 3)))
        except ValueError:
            break
        except IndexError:
            if noConfFlag:
                dataseries.append(np.zeros(25*2))
            else:
                dataseries.append(np.zeros(25*3))
    return dataseries,i

In [35]:
labelNames=['rennen','gehen']
dataPaths=['data/output/rennen/','data/output/gehen/']
samplePaths=[]
data=[]
labels=[]
#collect paths to data
dirs0 = os.listdir(dataPaths[0])
for p in dirs0:
    samplePaths.append(dataPaths[0]+p+'0')
dirs1 = os.listdir(dataPaths[1])
for p in dirs1:
    samplePaths.append(dataPaths[1]+p+'1')

#shuffel data
random.shuffle(samplePaths)
#print(samplePaths)
minLength = 75
for smp in samplePaths:
    l = smp[-1:]
    smp = smp[:-1]
    td, i = readData(smp)
    #if i<minLength:
    #    minLength = i
    data.append(td[:minLength])
    if l == '0':
        labels.append(np.array([1,0]))
    elif l == '1':
        labels.append(np.array([0,1]))
    else:
        #labels.append(np.array([0,0]))
        print("can't load: ",smp)
    print('.', end="")
        
print("data loaded.")
print("min Length: ", minLength)

.............................................data loaded.
min Length:  75


In [33]:
data

[[array([ 14.8523 , 247.168  ,   3.73959, 266.688  ,   3.74746, 265.35   ,
           0.     ,   0.     ,   0.     ,   0.     ,  13.4977 , 266.665  ,
           0.     ,   0.     ,   0.     ,   0.     ,   3.7305 , 331.96   ,
           3.74775, 333.245  ,   3.73425, 371.152  ,   0.     ,   0.     ,
          14.8127 , 330.662  ,  18.7983 , 375.022  ,  16.1916 , 423.318  ,
          12.2077 , 245.773  ,  14.8815 , 245.775  ,   3.73286, 245.79   ,
           0.     ,   0.     ,  38.3346 , 431.082  ,  35.7535 , 425.927  ,
          14.8033 , 431.075  ,   0.     ,   0.     ,   0.     ,   0.     ,
           0.     ,   0.     ]),
  array([ 18.7934 , 247.145  ,   3.72789, 266.69   ,   3.74556, 265.352  ,
           0.     ,   0.     ,   0.     ,   0.     ,  17.4634 , 267.984  ,
           0.     ,   0.     ,   0.     ,   0.     ,   3.0866 , 331.973  ,
           0.     ,   0.     ,   0.     ,   0.     ,   0.     ,   0.     ,
          16.2118 , 330.665  ,  21.3928 , 377.63   ,  17.4723 , 423

In [36]:
#define constants

time_steps=minLength
#hidden LSTM units
num_units=128
# 25*3=75; 25*2=50
n_input=50
#learning rate for adam
learning_rate=0.001

n_classes=2
#size of batch
batch_size=1

In [37]:
#shape for LSTM
d = np.concatenate(data, axis=0).reshape((-1,time_steps*50))
l = np.concatenate(labels, axis=0).reshape((-1,2))

In [38]:
print(d.shape,l.shape)
print(d)
print(l)

(45, 3750) (45, 2)
[[  3.73745 268.08      3.74878 ... 449.456   200.192   444.314  ]
 [  0.        0.        0.      ... 432.389   515.861   431.083  ]
 [  0.        0.        0.      ... 392.014   484.759   390.726  ]
 ...
 [ 14.8523  247.168     3.73959 ... 476.793   329.396   467.672  ]
 [620.267   163.59    652.997   ... 420.69    114.016   423.289  ]
 [  0.        0.        0.      ... 380.315   318.99    392.051  ]]
[[0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]


### TODO:
- Normalize/Standerdize

In [39]:
#split data in train and testset
testSize=10 
test_data = d[:testSize,:]
test_labels = l[:testSize,:]
train_data = d[testSize:,:]
train_labels = l[testSize:,:]

In [40]:
test_data.shape, test_labels.shape

((10, 3750), (10, 2))

In [41]:
train_data.shape, train_labels.shape

((35, 3750), (35, 2))

In [42]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [43]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

In [44]:
#defining the network
lstm_layer=tf.nn.rnn_cell.LSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

In [45]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [46]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [47]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<=30:
        batch_x=train_data[batch_size*iter:batch_size*(iter+1)]

        batch_y=train_labels[batch_size*iter:batch_size*(iter+1)]
        #print(batch_x)
        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %5==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1

    #calculating test accuracy
    test_data = test_data[:batch_size].reshape((-1, time_steps, n_input))
    test_label = test_labels[:batch_size]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

For iter  5
Accuracy  1.0
Loss  0.0
__________________
For iter  10
Accuracy  1.0
Loss  3.576278e-07
__________________
For iter  15
Accuracy  1.0
Loss  2.0146166e-05
__________________
For iter  20
Accuracy  1.0
Loss  1.7642818e-05
__________________
For iter  25
Accuracy  1.0
Loss  6.079655e-06
__________________
For iter  30
Accuracy  1.0
Loss  0.0
__________________
Testing Accuracy: 1.0


In [48]:
sess.close()

# Visualisation

In [49]:
show_data = d.reshape(-1,n_input)
show_label = [np.where(r==1)[0][0] for r in l]

In [50]:
#show_label

In [51]:
show_data.shape

(3375, 50)

In [52]:
testing = show_data[1]
testing[1]

270.685

In [55]:
import pygame
import sys, traceback

# Initialize Pygame.
pygame.init()
# Set size of pygame window.
screen=pygame.display.set_mode((900,750))
# Create empty pygame surface.
background = pygame.Surface(screen.get_size())
# Fill the background white color.
background.fill((255, 255, 255))
# Convert Surface object to make blitting faster.
background = background.convert()
# Copy background to screen (position (0, 0) is upper left corner).
screen.blit(background, (0,0))
# Create Pygame clock object.
clock = pygame.time.Clock()

mainloop = True
# Desired framerate in frames per second. Try out other values.
FPS = 30
# How many seconds the "game" is played.
playtime = 0.0

count = 0.0

speed = 1

try:
    
    while mainloop:
        # Do not go faster than this framerate.
        milliseconds = clock.tick(FPS*speed)
        playtime += milliseconds / 1000.0

        for event in pygame.event.get():
            # User presses QUIT-button.
            if event.type == pygame.QUIT:
                mainloop = False
            elif event.type == pygame.KEYDOWN:
                # User presses ESCAPE-Key
                if event.key == pygame.K_ESCAPE:
                    mainloop = False

        # Print framerate and playtime in titlebar.
        currentSample=int(count/n_input)%len(show_label)
        text = "FPS: {0:.2f}   Playtime: {1:.2f}  NR: {2:d}  Label: ".format(clock.get_fps(), playtime, currentSample+1)
        text += labelNames[show_label[currentSample]]
        pygame.display.set_caption(text)
        background.fill((255, 255, 255))
        f = show_data[int(count)%(show_data.shape[0])]
        for i in range(0,f.shape[0],2):#2 for no conf

            #confColor = 255 - int(255*f[i+2])%255
            #pygame.draw.rect(background, (confColor, confColor, confColor), (int(f[i]),int(f[i+1]),4,4)) #bw
            #pygame.draw.rect(background, (confColor, 0, 255-confColor), (int(f[i]),int(f[i+1]),4,4)) #rb
            pygame.draw.rect(background, (0, 0, 0), (int(f[i]),int(f[i+1]),4,4)) #no Conf
        #screen.blit(background, (300-f[8*3],300-f[8*3+1]))
        screen.blit(background, (0,0))
        #Update Pygame display.
        pygame.display.update()

        count += 1
except TypeError:
    print("Type error: ", count," ", i)
except:
    traceback.print_exc(file=sys.stdout)
else:
    pygame.quit()
# Finish Pygame.
pygame.quit()

# At the very last:
print("This game was played for {0:.2f} seconds".format(playtime))

This game was played for 2.35 seconds


# Live Data-read

In [25]:
import sys
import time
import logging
from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler
from watchdog.events import PatternMatchingEventHandler  

In [26]:
class MyHandler(PatternMatchingEventHandler):
    patterns = ["*.json"]
    data = []

    def process(self, event):
        """
        event.event_type 
            'modified' | 'created' | 'moved' | 'deleted'
        event.is_directory
            True | False
        event.src_path
            path/to/observed/file
        """
        # the file will be processed there
        try:
            #print(i)
            f = pd.read_json(event.src_path).iloc[0,0]['pose_keypoints_2d']
            self.data.append(f)
            print(f)
        except ValueError:
            print("ERROR reading: ",event.src_path, event.event_type)


    def on_created(self, event):
        self.process(event)
        
observer = Observer()
observer.schedule(MyHandler(), '.')
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()